In [ ]:
import pickle as pkl
import numpy as np
import os

root = '/home/lioruzan/latest_checkpoints'
runs = [[] for j in range(8)]
for i in range(8):
    if i==6: continue # nothing 
    r = os.path.join(root,'dir%d'%i)
    for j in range(20):
        p = os.path.join(r,'results_{}.pkl'.format(j))
        with open(p,'rb') as f:
            runs[i].append(pkl.load(f))

In [ ]:
# format of pickled files: {'gen_data': ( sample_x, x, m, sample_prob, colored_x ), 'size': size}
samps = []
datas = []
masks = []
colrs = []
probs = []
# collect generates samples from all 8 models for each experiment (20 total)
for j in range(20):
    samp = np.zeros((0,32,32,1),dtype=np.float32)
    data = samp.copy()
    mask = samp.copy().astype(np.int32)
    colr = np.zeros((0,32,32,3),dtype=np.float32)
    prob = np.zeros(len(data), dtype=np.float32)
    for i in range(8):
        if i==6: continue
        gen_data = runs[i][j]['gen_data']
        cutoff = runs[i][j]['size']
        total = len(gen_data) * len(gen_data[0][0])
        e = total-cutoff
        c = 1
        for sample_x, x, m, sample_prob, colored_x in gen_data:
            if c == len(gen_data):
                sample_x = sample_x[:-e]
                x = x[:-e]
                m = m[:-e]
                colored_x = colored_x[:-e]
                sample_prob = sample_prob[:-e]
            samp = np.vstack([samp, sample_x])
            data = np.vstack([data, x])
            mask = np.vstack([mask, m])
            colr = np.vstack([colr, colored_x])
            prob = np.concatenate([prob, sample_prob])
            c += 1
    samps.append(samp)
    datas.append(data)
    masks.append(mask)
    colrs.append(colr)
    probs.append(prob)

In [ ]:
average_psnrs=[]
std_psnrs=[]
for run in range(20):
    # calculate mean average psnr
    mses = []
    sample_x = samps[run]
    x = datas[run]
    # calculate per-picture psnr vectorized
    #change to 0..255
    a = np.round(127.5 * sample_x + 127.5)
    b = x
    mse = np.sum( np.power(a-b,2), axis=(1,2,3) ) / np.prod( a.shape[1:] ) # ignore batch size
    
    # discard all samples from padding
    
    psnrs = 20 * ( np.log10(255) - np.log10( np.sqrt(mse) ) )
    psnr_avg, psnr_std = np.mean(psnrs), np.std(psnrs)
    print("average psnr run {}: {}, std: {}".format(run, psnr_avg, psnr_std))
    average_psnrs.append(psnr_avg)
    std_psnrs.append(psnr_std)
    
# show stats summary
if len(average_psnrs)>0:
    print("mean average psnr: {}, std over averages: {}, mean psnr std: {}, std over stds: {}".format(
            np.mean(average_psnrs), np.std(average_psnrs),
            np.mean(std_psnrs), np.std(std_psnrs)))

In [1]:
''' OLD VERSION '''
import pickle as pkl
import numpy as np
import os.path as osp
import os

''' stitch together 4-model adaptive rotation results '''

root = '/home/lioruzan/checkpoints'
runs= [[] for j in range(4)]
for i in range(4):
    r = os.path.join(root,str(i))
    for j in range(10):
        p = os.path.join(r,'results_{}.pkl'.format(j))
        with open(p,'rb') as f:
            runs[i].append(pkl.load(f))
            

runss=[]
for j in range(10):
    samp=np.zeros((0,32,32,3))
    data=samp.copy()
    mask=data.copy()
    for i in range(4):
        for sample,(x,m) in runs[i][j]:
            sample = np.rot90(sample, k=-i, axes=(1,2))
            x = np.rot90(x, k=-i, axes=(1,2))
            m = np.rot90(m, k=-i, axes=(1,2))
            samp = np.vstack([samp,sample])
            data = np.vstack([data,x])
            mask = np.vstack([mask,m])
    runss.append((samp,data,mask))


In [4]:

''' calculate mean average psnr (+- mean average std)'''
average_psnrs, std_psnrs = [], []
for o, data, _ in runss:
    psnrs=[]
    for i in range(o.shape[0]):
        #change to 0..255
        x = np.round(127.5 * o[i] + 127.5)
        y = data[i]
        #mse
        mse = np.sum( np.power(x-y,2) ) / np.prod( x.shape )
        #psnr
        psnr = 20 * ( np.log10(255) - np.log10(np.sqrt(mse)) )
        psnrs.append(psnr)
        
    psnr_avg, psnr_std = np.mean(psnrs), np.std(psnrs)
    average_psnrs.append(psnr_avg)
    std_psnrs.append(psnr_std)
print('{:.5} +-{:.5}'.format(np.mean(average_psnrs), np.mean(std_psnrs)))


13.317 +-4.3397


In [ ]:

''' visualize results '''
p=np.random.randint(140)
plt.imshow(runss[0][1][p]/127.5-1)
plt.show()
plt.imshow((runss[0][1][p]/127.5-1)*runss[0][2][p])
plt.show()
plt.imshow(runss[0][0][p])
plt.show()